In [1]:
import pandas as pd

# Program creates dict with 4 DataFrames: 

In [2]:
years = [2013,2014,2015,2016]
d_frames = {}
for year in years:
    file_path = '{}.xlsx'.format(year)
    d_frames[year] = pd.read_excel(file_path)
    d_frames[year]['Year'] = year

In [3]:
d_frames[2013].head()

,country iso code,city_name,station_european_code,type_of_station,station_type_of_area,component_caption,statistic_value (µg/m3),above_DLV?,Year
0,AT,Klagenfurt,AT2KA21,Traffic,urban,PM10,42.321250,0,2013
1,AT,Klagenfurt,AT2KA71,Background,urban,PM10,39.242917,0,2013
2,AT,Linz,AT4S184,Background,urban,PM10,41.000000,0,2013
3,AT,Linz,AT4S416,Background,urban,PM10,45.000000,0,2013
4,AT,Linz,AT4S431,Traffic,urban,PM10,50.000000,0,2013


In [4]:
d_frames[2014].head()

,Country_ISO,City_name,Station_EoI_Code,Station_name,TypeOfMeasurementStation,AreaOfMeasurementStation,AirPollutant,AirPollutionLevel (µg/m3),Latitude,Longitude,Altitude,Year
0,AT,Graz,AT60171,Graz Ost Petersgasse,Traffic,urban,PM10,51.038625,47.059444,15.466125,362,2014
1,AT,Linz,AT4S431,Linz Römerberg B139,Traffic,urban,PM10,48.000000,48.303055,14.283070,262,2014
2,AT,Graz,AT60164,Graz Don Bosco,Traffic,urban,PM10,47.800000,47.055553,15.417789,358,2014
3,AT,Linz,AT4S415,Linz 24er Turm A7,Traffic,urban,PM10,46.700375,48.323334,14.297792,255,2014
4,AT,Wien,AT90TAB,Taborstraße,Traffic,urban,PM10,46.154000,48.218056,16.381958,160,2014


In [5]:
d_frames[2015].head()

,Country,City,AirQualityStationEoICode,AQStationName,AirPollutant,AirPollutionLevel,UnitOfAirpollutionLevel,AirQualityStationType,AirQualityStationArea,Longitude,Latitude,Altitude,Year
0,Albania,NaN,AL0205A,Durres,PM10,33.502958,µg/m3,Traffic,urban,19.449200,41.319900,1.0,2015
1,Albania,NaN,AL0206A,Shkoder,PM10,33.462417,µg/m3,Background,urban,19.523420,42.313900,13.0,2015
2,Albania,NaN,AL0204A,Vlora,PM10,33.314208,µg/m3,Background,urban,19.486200,40.403090,25.0,2015
3,Andorra,NaN,AD0942A,Fixa,PM10,45.041667,µg/m3,Background,urban,1.539138,42.509694,1080.0,2015
4,Austria,Linz,AT4S431,Linz Römerberg B139,PM10,46.611875,µg/m3,Traffic,urban,14.282778,48.302778,262.0,2015


In [6]:
d_frames[2016].head()

,Country,City,AirQualityStationEoICode,AQStationName,AirPollutant,AirPollutionLevel,UnitOfAirpollutionLevel,AirQualityStationType,AirQualityStationArea,Longitude,Latitude,Altitude,Year
0,Montenegro,NaN,ME0001A,Pljevlja,PM10,228.280250,ug/m3,Background,urban,19.358101,43.355034,777.0,2016
1,Turkey,NaN,TR760141,IGDIR,PM10,220.916667,ug/m3,Background,urban,44.053610,39.926110,859.0,2016
2,Turkey,NaN,TR810141,DUZCE,PM10,206.125000,ug/m3,Background,urban,31.148050,40.845550,144.0,2016
3,The former Yugoslav Republic of Macedonia,NaN,MK0031A,Lisice,PM10,199.753737,ug/m3,Industrial,urban,21.470000,41.978330,235.0,2016
4,Turkey,NaN,TR380131,KAYSERI-HURRIYET,PM10,188.791667,ug/m3,Traffic,urban,35.469610,38.723550,1056.0,2016


# Extracting columns of intrest, and unifing column names:

In [7]:
d_frames[2013] = d_frames[2013][['station_european_code','station_type_of_area','statistic_value (µg/m3)','Year']]
d_frames[2014] = d_frames[2014][['Station_EoI_Code','AreaOfMeasurementStation','AirPollutionLevel (µg/m3)','Year']]
d_frames[2015] = d_frames[2015][['AirQualityStationEoICode','AirQualityStationArea','AirPollutionLevel','Year']]
d_frames[2016] = d_frames[2016][['AirQualityStationEoICode','AirQualityStationArea','AirPollutionLevel','Year']]

In [8]:
d_frames[2013].columns = d_frames[2016].columns
d_frames[2014].columns = d_frames[2016].columns
d_frames[2015].columns = d_frames[2016].columns

## Concatinating all 4 DF from dictionary with pd.concat. With unified columns names no extra argument needs to be pased:

In [9]:
pm10_data = pd.concat(d_frames, sort=False)
pm10_data.head()

AirQualityStationEoICode AirQualityStationArea  AirPollutionLevel  Year
2013 0                  AT2KA21                 urban          42.321250  2013
     1                  AT2KA71                 urban          39.242917  2013
     2                  AT4S184                 urban          41.000000  2013
     3                  AT4S416                 urban          45.000000  2013
     4                  AT4S431                 urban          50.000000  2013

## Extracting country abbrevation with accesing string atribiutes of one of the column. 

In [10]:
pm10_data['Country']=pm10_data['AirQualityStationEoICode'].str[:2]

## Setting MultiIndexes 

In [11]:
pm10_data = pm10_data.set_index(['Year','AirQualityStationEoICode'])

In [12]:
pm10_data.head()

AirQualityStationArea  AirPollutionLevel Country
Year AirQualityStationEoICode                                                 
2013 AT2KA21                                  urban          42.321250      AT
     AT2KA71                                  urban          39.242917      AT
     AT4S184                                  urban          41.000000      AT
     AT4S416                                  urban          45.000000      AT
     AT4S431                                  urban          50.000000      AT